# Assignment 1: ODrive setup and initial kinematics exploration

In homework 1 we will learn the basic functionality of the ODrive motor controller and the kinematics of the 5-bar leg. There are 5 problems below. You will turn this assignment in by uploading it to your own github repository for this assignment and then sending a link to ngravish@eng.ucsd.edu with the subject MAE_207_HW1. The assignment is to be completed in pairs, one per team.

If you would like, you directly ```fork``` this repository into your github account, this is found in the upper right hand corner. You can then download the repository to your computer using the ```clone``` button on the right side. Once you are done with the assignment, commit the changes back to the github online repository. 

Alternatively you can just download the .ipynb file, edit it as you solve the problems, and then in the end upload to a github repo.

Team #:

Names:



## Problem 1: Familiarization with ODrive firmware

__1.1__ You should follow the instructions to flash the motor controller firmware found [here](https://github.com/madcowswe/ODrive).

__1.2__ In the ```Odrive/Firmware/MotorControl``` folder are the code files that control the motor. The most important file is ```low_level.c```. Find this file and familiarize yourself with it. 

__1.3__ In ```low_level.c``` the main function for motor control is ```control_motor_loop(Motor_t* motor)```. Find this function and copy and paste blocks of it into the cell below.  Annotate the function step-by-step in simple language the operations it performs. (*I have started this annotation*). Use the three backticks \`\`\` and the letter c for the c language to highlight each code block.






```c
void control_motor_loop(Motor_t* motor) {
    while (*(motor->axis_legacy.enable_control)) {
        if (osSignalWait(M_SIGNAL_PH_CURRENT_MEAS, PH_CURRENT_MEAS_TIMEOUT).status != osEventSignal) {
            motor->error = ERROR_FOC_MEASUREMENT_TIMEOUT;
            break;
        }

        if (!do_checks(motor))
            break;
        if (!loop_updates(motor))
            break;
```

The above code initiates a loop that will continue as long as the enable_control flag is true. The first three if statements check for errors in the motor current measurement, and check if there are errors in the motor operation.

```c
float vel_des = motor->vel_setpoint;
        if (motor->control_mode >= CTRL_MODE_POSITION_CONTROL) {
            if (motor->rotor_mode == ROTOR_MODE_SENSORLESS) {
                motor->error = ERROR_POS_CTRL_DURING_SENSORLESS;
                break;
            }
            float pos_err = motor->pos_setpoint - motor->encoder.pll_pos;
            vel_des += motor->pos_gain * pos_err;
        }
        
```
The above code corresponds to position control. The first two if statements make sure that control mode with position can be used. If the rotor mode is not sensed it will produce an error.

```c
float vel_lim = motor->vel_limit;
        if (vel_des > vel_lim) vel_des = vel_lim;
        if (vel_des < -vel_lim) vel_des = -vel_lim;
        
```
The above code sets the velocity limits. The if statements make sure that the velocity does not go beyond the limited velocity.

```c
float Iq = motor->current_setpoint;

```
The above code controls the velocity with the current. 

```c
if (motor->anticogging.use_anticogging) {
            Iq += motor->anticogging.cogging_map[mod(motor->encoder.pll_pos, motor->encoder.encoder_cpr)];
        }

        float v_err = vel_des - get_pll_vel(motor);
        if (motor->control_mode >= CTRL_MODE_VELOCITY_CONTROL) {
            Iq += motor->vel_gain * v_err;
        }
```
The above code uses current positions to apply a current feed-forward. Sometimes the encoder will give negative positions and the if statements make sure that this is interpreted correctly. 

```c
 Iq += motor->vel_integrator_current;
 
```
 
The above code checks the current before limiting velocity 
```c
   if (motor->rotor_mode == ROTOR_MODE_ENCODER ||
            motor->rotor_mode == ROTOR_MODE_RUN_ENCODER_TEST_SENSORLESS) {
            Iq *= motor->encoder.motor_dir;
        }
```
The above code uses an if statemtent to check the direction of the encoder and correct it .

```c
 float Ilim = MACRO_MIN(motor->current_control.current_lim, motor->current_control.max_allowed_current);
        bool limited = false;
        if (Iq > Ilim) {
            limited = true;
            Iq = Ilim;
        }
        if (Iq < -Ilim) {
            limited = true;
            Iq = -Ilim;
        }
```
The above code limits the current. The if statements check to see if the current is greater than the limited amount and sets it back to the limited. 

```c
   if (motor->control_mode < CTRL_MODE_VELOCITY_CONTROL) {
        
            motor->vel_integrator_current = 0.0f;
        } else {
            if (limited) {
                
                motor->vel_integrator_current *= 0.99f;
            } else {
                motor->vel_integrator_current += (motor->vel_integrator_gain * current_meas_period) * v_err;
            }
        }
```
The above code uses if statements to control the velocity integrator.

```c

if (motor->motor_type == MOTOR_TYPE_HIGH_CURRENT) {
            if(!FOC_current(motor, 0.0f, Iq)){
                break; // in case of error exit loop, motor->error has been set by FOC_current
            }
        } else if (motor->motor_type == MOTOR_TYPE_GIMBAL) {
            //In gimbal motor mode, current is reinterptreted as voltage.
            if(!FOC_voltage(motor, 0.0f, Iq)){
                break; // in case of error exit loop, motor->error has been set by FOC_voltage
            }
        } else {
            motor->error = ERROR_NOT_IMPLEMENTED_MOTOR_TYPE;
            break;
        }

        ++(motor->loop_counter);
    }
```
The above code initiates the command controlled by the current. The if statements stops the loop if there is an error in the current. If not, then it will check the voltage for gimbal controlled motors and stop if there is an error in the voltage.

## Problem 2: Connecting to ODrive through the python library

__2.1__ The ODrive provides a python library called ```odrive``` in the ```/ODrive/Tools/``` directory. A python library consists of a folder (```odrive``` in this case) which contains python files and a file called ```__init__.py```. Find this library.

__2.2__ Test that you can import the ```odrive``` library and other required libraries into Python by running the code below (shift-enter when the cell is selected). You may need to move your Copy the ```odrive``` folder to the same folder as this notebook, or move this notebook to the same directory as ```odrive```. 



In [1]:
import odrive.core
import time
import math

import numpy as np
import matplotlib.pyplot as plt

from IPython.lib.display import YouTubeVideo # for youtube videos

from IPython.display import Image


# This may be qt4 depending on your python environment
%matplotlib qt5


__2.3__ Run the code below to connect to the odrive motor controller. If this executes successfully you will be able to access the motor controller functions through the ```my_drive``` variable.

In [270]:
my_drive = odrive.core.find_any(consider_usb=True, consider_serial=False, printer=print)

# define handles to the motor functions
m0=my_drive.motor0
m1=my_drive.motor1 


looking for ODrive...
Found ODrive via PyUSB
ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,1
		EndpointAddress 1
		EndpointAddress 129

EndpointAddress for writing 1

EndpointAddress for reading 129

Connecting to device on USB device bus 0 device 20
JSON: [{"name":"","id":0,"type":"json","access":"rw"},{"name":"vbus_voltage","id":1,"type":"float","access":"r"},{"name":"serial_number","id":2,"type":"uint64","access":"r"},{"name":"run_anticogging_calibration","id":3,"type":"function","arguments":[]},{"name":"config","id":5,"type":"object","members":[{"name":"brake_resistance","id":6,"type":"float","access":"rw"}]},{"name":"axis0","id":8,"type":"object","members":[{"name":"config","id":9,"type":"object","members":[{"name":"enable_control_at_start","id":10,"type":"bool","access":"rw"},{"name":"do_calibration_at_start","id":11,"type":"bool","access":"rw"}]}]},{"name":"motor0","id":14,"type":"object","members":[{"name":"config","id":15,"type":"object","m

In [41]:
def cur_run(a):
    m0.set_current_setpoint(a)
    m1.set_current_setpoint(a)

In [42]:
def cur_pos(a):
    m0.set_pos_setpoint(a,0,0)
    m1.set_pos_setpoint(a,0,0)

## Problem 3: Exploring and interacting with the ODrive

__3.1__ Write a function to call read the encoder position and velocity estimates and return them. As a start, the encoder position for motor 0 can be read using the ```m0.encoder.pll_pos,m1.encoder.pll_pos``` command. 

In [271]:
# in python we can define a function using the def command. The function definition code is indicated by tab indented code. When the 
# tab indentation stops the function definition has stopped. As shown below we can define a function and then call it immediately after
def get_encoder_state():
    
    #
    # Your code here
    #
    motor0_position=m0.encoder.pll_pos
    motor1_position=m1.encoder.pll_pos
    motor0_velocity=m0.encoder.pll_vel
    motor1_velocity=m1.encoder.pll_vel
    return (motor0_position, motor0_velocity, motor1_position, motor1_velocity)
    
    
# We can now call this function in the same code block
state = get_encoder_state()
print(state)


(-0.6061868667602539, -0.0851898193359375, -0.09139205515384674, 0.1612396240234375)


__3.2__ The motor encoder returns absolute position measurements that are zerod on the position the motor was in during startup. It is useful to define new motor origins for the left and righ motor. Extend the legs so that they are straight and then read the encoder positions to the tuple variable 

```home_position = (motor0_home, motor1_home)```

A tuple is like an array, it can contain sequences of objects (numbers, strings, etc.), but it cannot be changed after it is written so it is perfect for storing unchangeable data like the home location.

In [272]:
#
#  Your code here
#
motor0_home=state[0]
motor1_home=state[2]

home_position = (motor0_home,motor1_home)# your code here
print(home_position)


(-0.6061868667602539, -0.09139205515384674)


In [221]:
m0.config.pos_gain=30
m1.config.pos_gain=30

__3.3__ The motor move commands will move them to absolute positions with respect to the encoder zero position, this may not be the same as the ```home_position``` that we defined above. Write a function to move the motors to an absolute position relative to the ```home_position```. For reference we can move a motor with the ```set_pos_setpoint(pos_setpoint, vel_feed_forward, current_feed_forward)``` command. We don't need the feedforward terms in this problem.

In [231]:
def move_motors(motor0_new_position, motor1_new_position, home):
    #
    # Your code here
    #
    a=motor0_new_position+home[0]
    b=motor1_new_position+home[1]
    m0.pos_setpoint=a
    m1.pos_setpoint=b
# we can call the function here 
move_motors(0, 0, home_position) # note make the intial move distance small!!

## Problem 4: Determine the workspace of the robot leg

__4.1__ The end of the robot leg has a hole in it. Place a pen in the hole, place a paper below the robot and trace out the boundary of the robot leg's workspace by hand. Make sure to note where the edge of the aluminum frame is. Take a picture of the workspace tracing (keep your robot in place) and attach it to you this notebook. You can attach images by putting them in the same directory and then writing ```![Image description](filename.extension)```  


#  Image for 4.1

<img src="problem4.1.jpg">

__4.2__ Now repeat the measurement of the leg workspace while recording the encoder position (*relative to your new home*) in a loop. This is a measurement configuration space of the robot which is two dimensional ($\theta_0$, $\theta_1$), and bounded by the leg range of motion. Make a plot of the configuration space boundaries by plotting your measured positions as ```xy``` coordinates ($\theta_0$, $\theta_1$). Include two plots below, the first is the configuration space using encoder units, the second is scaled to $-\pi,\pi$. You will have to determined the calibration constant (note the encoder generates 2048\*4 counts per revolution).

In [160]:
# Setup an array to store the encoder positions
left_motor = [] # initialize a empty list. You can add to a list with the left_motor.append(item) command
right_motor = [] # initialize an empty list
left_motor1 = []
right_motor1 = []
calibration=4096/(2*math.pi)
#different amplitude and phase combinations of your leg gait.    
t1=time.monotonic()
t2=0
while t2<15:
    t3=time.monotonic()
    t2=t3-t1
    
    motor0_position=m0.encoder.pll_pos-motor0_home
    motor1_position=m1.encoder.pll_pos-motor1_home
    left_motor.append(motor0_position)
    right_motor.append(motor1_position)
    left_motor1.append(motor0_position/calibration)
    right_motor1.append(motor1_position/calibration)

    #current_time = time.monotonic()
    


    
# Plot the results using encoder values (centered on your new home) and scaled to be between -pi, pi


plt.figure()
# keep a reference to the first axis
ax1 = plt.subplot(1,2,1)
ax1.plot(left_motor, right_motor)
plt.xlabel('Left')
plt.ylabel('Right')

# and a reference to the second axis
ax2 = plt.subplot(1,2,2)
ax2.plot(left_motor1, right_motor1)
plt.xlabel('Left')
plt.ylabel('Right')



Text(0,0.5,'Right')

# Image for Problem 4.2

<img src="problem4.2.png">

## Problem 5: Command a simple cyclic gait of the leg

__5.1__ In this last problem we will command a simple cyclic gait of the leg by sending sinusoidal position commands to the motors and exploring how phase and amplitude differences influence the foot motion. Record the encoder position while running the gait and provide plots of the motor angles as a function of time. Additionally, take a video of each gait and post it to youtube, including a link in the notebook at the very bottom. You can include youtube videos by entering the command ```YouTubeVideo('video_code')``` in a code cell, where video code is found from the youtube page.

*Provide plots and videos for three different gaits*


In [347]:
Amplitude_left = 1500 # your code
Amplitude_right = 600 # your code
Phase_left = 30 # your code
Phase_right = 20# your code
calibration=4096/(2*math.pi)

left_motor_g = []
right_motor_g = []

m0.config.pos_gain=20
m1.config.pos_gain=20

initial_time = time.monotonic()  # record the current time
current_time = time.monotonic() 

while (current_time - initial_time) < 10:
    #
    #   Your code -- send position command
    #   
    #
    #   Your code -- read encoder variables and save them to a list 
    #
    current_time = time.monotonic()
    a=Amplitude_left*math.sin(10*current_time+Phase_left)
    b=Amplitude_right*math.sin(10*current_time+Phase_right)
    m0.pos_setpoint=a
    m1.pos_setpoint=b
    
    motor0_position_g=m0.encoder.pll_pos-motor0_home
    motor1_position_g=m1.encoder.pll_pos-motor1_home
    left_motor_g.append(motor0_position_g/calibration)
    right_motor_g.append(motor1_position_g/calibration)
    
    time.sleep(0.01)


In [343]:
#
#  Plot for gait 1
# gait 1 

# Amplitude_left = 1000,Amplitude_right = 600,Phase_left = 20,Phase_right = 20

# we can use the same code for three different gait
plt.figure()
# keep a reference to the first axis
ax1 = plt.subplot(1,2,1)
ax1.plot(left_motor_g, right_motor_g)
plt.xlabel('Left')
plt.ylabel('Right')
plt.title('gait_3')

Text(0.5,1,'gait_3')

In [345]:
m0.config.pos_gain=0
m1.config.pos_gain=0


#  Plot for gait 1
# Amplitude_left = 1000,Amplitude_right = 600,Phase_left = 20,Phase_right = 20

<img src="gait1.png">


#  Youtube video for gait 1


YouTubeVideo('https://www.youtube.com/watch?v=csM4kfCjaco')


#  Plot for gait 2
# Amplitude_left = 1000,Amplitude_right = -600,Phase_left = 10,Phase_right = 20

<img src="gait2.png">


#  Youtube video for gait 2


YouTubeVideo('https://www.youtube.com/watch?v=sp60LBYDdnI')


#  Plot for gait 3
#Amplitude_left = 1500,Amplitude_right = 600,Phase_left = 20,Phase_right = 20

<img src="gait3.png">


#  Youtube video for gait 3


YouTubeVideo('https://www.youtube.com/watch?v=6Bk11FVdhAk')
